In [1]:
from sym_so3 import *
from drake_sympy import *
import debug

Using the standard formulas:

Let $r = [r_r, r_p, r_y]$ be the Euler angles, and look at derivative values

\begin{align}
    \text{Skew symmetric angular velocity matrix:} \\
    \hat{\omega} &= R^T \dot{R}   \\
    \text{Skew symmetric angular acceleration matrix:} \\
    \hat{\alpha} &= \dot{R}^T \dot{R} + R^T \ddot{R} \\
        &= \hat{\omega}^T \hat{\omega} + R^T \ddot{R} \\
    \text{Alternatively, can use time derivative of (unskewed) angular velocity} \\
    \alpha &= \dot{\omega} = \frac{\partial \omega}{\partial r} \dot{r} + \frac{\partial \omega}{\partial \dot{r}} \ddot{r}
\end{align}

As shown below, I (Eric) am having trouble making the skew-symmetric angular velocity matrix actually work when rotation occurs on more than one axis

In [2]:
workspace = SecondOrderWorkspace.make(["r", "p", "y"])

r_s, rd_s, rdd_s = workspace.syms

# Compute rotation matrix using rpy.
R_s = RollPitchYaw_[Expression](r_s).ToRotationMatrix().matrix()

# Compute first- and second-derivatives w.r.t. time.
Rd_s, Rdd_s = derivatives_2nd(R_s, r_s, rd_s, rdd_s)
# Compute skew-symmetric angular velocity matrix.
wh_s = R_s.T @ Rd_s

# Compute angular accel. directly from angular velocity.
ah_s = derivatives_1st(wh_s, cat(r_s, rd_s), cat(rd_s, rdd_s))

# Try using chain rule of skew-symmetric relationship.
# WARNING: While this chain rule seems correct, numerically and
# symbolically it seems incorrect - see below :(
# ah_s_bad = Rd_s.T @ Rd_s + R_s.T @ Rdd_s
ah_s_bad = wh_s.T @ wh_s + R_s.T @ Rdd_s

In [3]:
env = workspace.env
r_v, rd_v, rdd_v = workspace.values

# # Single-axis: This shows numerical values that are OK for both angular accelerations.
# r_v[:] = [0.0, 0.0, 0.4]
# rd_v[:] = [0.0, 0.0, 0.7]
# rdd_v[:] = [0.0, 0.0, 1.1]

# Multi-axis: This does *not* show good numerical values, which is reflected in symbolics.
r_v[:] = [0.2, 0.3, 0.4]
rd_v[:] = [0.5, 0.6, 0.7]
rdd_v[:] = [0.8, 0.9, 1.1]

R = sym.Evaluate(R_s, env)
wh = sym.Evaluate(wh_s, env)
w = unskew(wh)

ah = sym.Evaluate(ah_s, env)
a = unskew(ah)

ah_bad = sym.Evaluate(ah_s_bad, env)

In [4]:
# Decent-looking values.
print(R @ R.T)
print(w)
print(a)

[[ 1.00000000e+00 -5.54075129e-17  6.97071887e-17]
 [-5.54075129e-17  1.00000000e+00  2.18104550e-17]
 [ 6.97071887e-17  2.18104550e-17  1.00000000e+00]]
[0.29313586 0.72089719 0.53620376]
[0.07368645 1.33427893 0.36902733]


In [5]:
# Blech!
print(ah_bad)
print(ah_bad + ah_bad.T)

[[-3.33066907e-16 -1.08992452e+00  7.98075172e-01]
 [ 6.12316088e-01 -3.63114876e-01 -8.44495507e-01]
 [-1.38359600e+00  1.05360039e-01 -5.06134128e-02]]
[[-6.66133815e-16 -4.77608428e-01 -5.85520829e-01]
 [-4.77608428e-01 -7.26229753e-01 -7.39135468e-01]
 [-5.85520829e-01 -7.39135468e-01 -1.01226826e-01]]


In [6]:
to_sympy = make_drake_to_sympy(cat(r_s, rd_s, rdd_s))
r_sympy = [to_sympy[hash(x)] for x in r_s]

def pretty(A, *, simplify=True):
    A = drake_to_sympy_matrix(A, to_sympy)
    A = pretty_trig(A, r_sympy, simplify=simplify)
    return A

The `trigsimp` stuff will take a few min

For compactness, I substitute stuff like $cos(r)$ with $c_r$, $sin(y)$ with $s_y$, etc.

In [7]:
pretty(R_s)

Matrix([
[c_p*c_y, -c_r*s_y + c_y*s_p*s_r, c_r*c_y*s_p + s_r*s_y],
[c_p*s_y,  c_r*c_y + s_p*s_r*s_y, c_r*s_p*s_y - c_y*s_r],
[   -s_p,                c_p*s_r,               c_p*c_r]])

In [8]:
pretty(Rd_s)

Matrix([
[-c_p*s_y*ydot - c_y*pdot*s_p,  c_p*c_y*pdot*s_r + rdot*(c_r*c_y*s_p + s_r*s_y) - ydot*(c_r*c_y + s_p*s_r*s_y), c_p*c_r*c_y*pdot + rdot*(c_r*s_y - c_y*s_p*s_r) + ydot*(-c_r*s_p*s_y + c_y*s_r)],
[ c_p*c_y*ydot - pdot*s_p*s_y, c_p*pdot*s_r*s_y + rdot*(c_r*s_p*s_y - c_y*s_r) + ydot*(-c_r*s_y + c_y*s_p*s_r),  c_p*c_r*pdot*s_y - rdot*(c_r*c_y + s_p*s_r*s_y) + ydot*(c_r*c_y*s_p + s_r*s_y)],
[                   -c_p*pdot,                                                     c_p*c_r*rdot - pdot*s_p*s_r,                                                    -c_p*rdot*s_r - c_r*pdot*s_p]])

In [9]:
pretty(Rdd_s)

Matrix([
[-c_p*c_y*pdot**2 - c_p*c_y*ydot**2 - c_p*s_y*yddot - c_y*pddot*s_p,  c_p*c_y*pddot*s_r - c_y*pdot**2*s_p*s_r + rddot*(c_r*c_y*s_p + s_r*s_y) + rdot**2*(c_r*s_y - c_y*s_p*s_r) - yddot*(c_r*c_y + s_p*s_r*s_y) + ydot**2*(c_r*s_y - c_y*s_p*s_r),  c_p*c_r*c_y*pddot - c_r*c_y*pdot**2*s_p + rddot*(c_r*s_y - c_y*s_p*s_r) - rdot**2*(c_r*c_y*s_p + s_r*s_y) + yddot*(-c_r*s_p*s_y + c_y*s_r) - ydot**2*(c_r*c_y*s_p + s_r*s_y)],
[ c_p*c_y*yddot - c_p*pdot**2*s_y - c_p*s_y*ydot**2 - pddot*s_p*s_y, c_p*pddot*s_r*s_y - pdot**2*s_p*s_r*s_y + rddot*(c_r*s_p*s_y - c_y*s_r) - rdot**2*(c_r*c_y + s_p*s_r*s_y) + yddot*(-c_r*s_y + c_y*s_p*s_r) - ydot**2*(c_r*c_y + s_p*s_r*s_y), c_p*c_r*pddot*s_y - c_r*pdot**2*s_p*s_y - rddot*(c_r*c_y + s_p*s_r*s_y) + rdot**2*(-c_r*s_p*s_y + c_y*s_r) + yddot*(c_r*c_y*s_p + s_r*s_y) + ydot**2*(-c_r*s_p*s_y + c_y*s_r)],
[                                          -c_p*pddot + pdot**2*s_p,                                                                                     

In [10]:
pretty(ah_s)

Matrix([
[                                                                                 0, -c_p*c_r*yddot + c_r*pdot*s_p*ydot + pddot*s_r + rdot*(c_p*s_r*ydot + c_r*pdot), c_p*s_r*yddot + c_r*pddot - pdot*s_p*s_r*ydot + rdot*(c_p*c_r*ydot - pdot*s_r)],
[c_p*c_r*yddot - 1.0*c_r*pdot*s_p*ydot - pddot*s_r - rdot*(c_p*s_r*ydot + c_r*pdot),                                                                               0,                                              c_p*pdot*ydot - rddot + s_p*yddot],
[  -c_p*s_r*yddot - c_r*pddot + pdot*s_p*s_r*ydot + rdot*(-c_p*c_r*ydot + pdot*s_r),                                              -c_p*pdot*ydot + rddot - s_p*yddot,                                                                              0]])

Following is not skew symmetric?!

In [11]:
pretty(ah_s_bad)

Matrix([
[                                                                                 0, -c_p*c_r*yddot - c_p*rdot*s_r*ydot - c_r*pdot*rdot + c_r*pdot*s_p*ydot + pddot*s_r, -c_p*c_r*rdot*ydot + c_p*s_r*yddot + c_r*pddot + pdot*rdot*s_r - pdot*s_p*s_r*ydot],
[ c_p*c_r*yddot - c_p*rdot*s_r*ydot - c_r*pdot*rdot + c_r*pdot*s_p*ydot - pddot*s_r,                      ydot*(pdot*sin(p - 2*r)/2 - pdot*sin(p + 2*r)/2 - 2*rdot*s_p),                        -2*c_p*c_r**2*pdot*ydot + c_p*pdot*ydot - rddot + s_p*yddot],
[-c_p*c_r*rdot*ydot - c_p*s_r*yddot - c_r*pddot + pdot*rdot*s_r - pdot*s_p*s_r*ydot,                        -2*c_p*c_r**2*pdot*ydot + c_p*pdot*ydot + rddot - s_p*yddot,                     ydot*(-pdot*sin(p - 2*r)/2 + pdot*sin(p + 2*r)/2 - 2*rdot*s_p)]])